# AX=YB
This notebook shows the process of calibrating the extrinsic parameters between the Vicon marker coordinate frame installed on the end-effector of a robot with respect to the end-effector frame.

### Vicon Sensor

In [1]:
from SimpleHandEye.interfaces.ros2 import ROS2ExecutorManager, ROS2CameraReader, ROS2TFInterface
import rclpy
rclpy.init()    
executor_manager = ROS2ExecutorManager()
vicon_sensor = ROS2TFInterface('vicon/World', 'vicon/WHITE_FR3_EF/WHITE_FR3_EF', node_name='vicon_sensor_node')
executor_manager.add_node(vicon_sensor)
executor_manager.start()

In [2]:
vicon_sensor.get_pose()

array([[ 0.8237402 , -0.04721833, -0.5649978 , -1.96782698],
       [-0.15238737, -0.97829532, -0.14041494,  0.2420608 ],
       [-0.54610454,  0.20176396, -0.81305666,  1.44821424],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

### FR3 Forward Kinematic

In [3]:
 
from FR3Py.robot.interface import FR3Real
robot = FR3Real(robot_id='fr3')

Interface Running...


In [4]:
from FR3Py.robot.model import PinocchioModel
model = PinocchioModel()

In [6]:
import numpy as np
def getFK(robot, model):
    state = robot.getJointStates()
    q , dq = state['q'], state['dq']
    info = model.getInfo(q, dq)
    R_ee = info['R_EE']
    t_ee = info['P_EE']
    T = np.hstack([R_ee, t_ee.reshape(3,1)])
    T = np.vstack([T, np.array([0, 0, 0, 1])])
    return T

getFK(robot, model)


array([[ 9.99995667e-01,  2.53175935e-03, -1.50182585e-03,
         3.06750390e-01],
       [ 2.52655967e-03, -9.99990843e-01, -3.45409351e-03,
         7.10068705e-04],
       [-1.51055703e-03,  3.45028409e-03, -9.99992907e-01,
         4.86721704e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

## Instantiate the Solver

In [7]:
from SimpleHandEye.solvers import OpenCVSolver
solver = OpenCVSolver()

## Collect The Calibration Dataset

In [8]:
import ipywidgets as widgets
import numpy as np
from IPython.display import display
from pprint import pprint
from IPython.display import clear_output
np.set_printoptions(suppress=True, precision=3)

# The dataset
A_list = []
B_list = []
def on_sample_clicked(b):
    A = getFK(robot, model)
    B = vicon_sensor.get_pose()
    print("A=")
    pprint(A)
    print("B=")
    pprint(B)
    A_list.append(A)
    B_list.append(B)
    print("*************")

def on_compute_clicked(b):
    try:
        X,Y = solver.solve(A_list, B_list)
        clear_output(wait=True)
        print("X=")
        pprint(X)
        print("Y=")
        pprint(Y)
    except:
        print("Bad dataset, please record again")
        A_list.clear()
        B_list.clear()
        

sample_button = widgets.Button(description="Sample")
compute_button = widgets.Button(description="Compute")

sample_button.on_click(on_sample_clicked)
compute_button.on_click(on_compute_clicked)
display(sample_button)
display(compute_button)

X=
array([[ 0.813, -0.079, -0.577,  0.016],
       [ 0.187,  0.974,  0.131, -0.002],
       [ 0.551, -0.214,  0.807, -0.059],
       [ 0.   ,  0.   ,  0.   ,  1.   ]])
Y=
array([[ 0.999,  0.032,  0.002,  2.279],
       [-0.032,  0.999,  0.018, -0.326],
       [-0.002, -0.018,  1.   , -0.901],
       [ 0.   ,  0.   ,  0.   ,  1.   ]])


In [9]:
X, Y = solver.solve(A_list, B_list)

In [10]:
ef_T_marker = X
Base_T_viconW = Y

In [11]:
for i in range(len(A_list)):
    print((A_list[i]@X@np.linalg.inv(Y@B_list[i]))[:3,-1])

[-0.005  0.001  0.002]
[-0.004 -0.002  0.002]
[0.003 0.005 0.002]
[ 0.003  0.004 -0.004]
[ 0.003 -0.001 -0.002]
[ 0.003 -0.001 -0.002]
[-0.002 -0.    -0.001]
[-0.003 -0.001 -0.   ]
[ 0.001 -0.001 -0.002]
[-0.    -0.004 -0.001]


In [11]:
import pickle
# camera_name = 'side_left_cam'
dataset_name = f'fr3-vicon-dataset.pkl'
with open(dataset_name, 'wb') as f:
    data = {
        'A_list': A_list,
        'B_list': B_list,
        'X': X,
        'Y': Y,
        'ef_T_marker':ef_T_marker,
        'Base_T_viconW':Base_T_viconW
    }
    pickle.dump(data, f)